# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 10:30AM,246697,20,ALU0007951,Alumier Labs Inc.,Released
1,Sep 19 2022 10:30AM,246697,20,ALU0007952,Alumier Labs Inc.,Released
2,Sep 19 2022 10:30AM,246697,20,ALUR016613299,Alumier Labs Inc.,Released
3,Sep 19 2022 10:30AM,246697,20,ALUR024222476,Alumier Labs Inc.,Released
4,Sep 19 2022 10:30AM,246697,20,ALUR079065075,Alumier Labs Inc.,Released
5,Sep 19 2022 10:30AM,246697,20,ALUR091981099,Alumier Labs Inc.,Released
6,Sep 19 2022 10:30AM,246697,20,ALUR096368292,Alumier Labs Inc.,Released
7,Sep 19 2022 10:30AM,246697,20,ALUR155771389,Alumier Labs Inc.,Released
8,Sep 19 2022 10:30AM,246697,20,ALUR173504446,Alumier Labs Inc.,Released
9,Sep 19 2022 10:30AM,246697,20,ALUR247073459,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
5,246693,Released,1
6,246694,Released,10
7,246695,Released,3
8,246697,Released,19
9,246698,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246693,NaN,NaN,1.0
246694,NaN,NaN,10.0
246695,NaN,NaN,3.0
246697,NaN,NaN,19.0
246698,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246672,3.0,3.0,1.0
246677,0.0,1.0,0.0
246684,0.0,0.0,48.0
246693,0.0,0.0,1.0
246694,0.0,0.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246672,3,3,1
246677,0,1,0
246684,0,0,48
246693,0,0,1
246694,0,0,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246672,3,3,1
1,246677,0,1,0
2,246684,0,0,48
3,246693,0,0,1
4,246694,0,0,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246672,3,3,1
1,246677,,1,
2,246684,,,48
3,246693,,,1
4,246694,,,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.
19,Sep 19 2022 10:29AM,246698,10,Eye Pharma Inc
20,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc."
30,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc."
33,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America
34,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC"
35,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation
83,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,,,19
1,Sep 19 2022 10:29AM,246698,10,Eye Pharma Inc,,,1
2,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",,,10
3,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",,,3
4,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,,,1
5,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",,1,
6,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,,,48
7,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd",3,3,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,19,,
1,Sep 19 2022 10:29AM,246698,10,Eye Pharma Inc,1,,
2,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",10,,
3,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",3,,
4,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1,,
5,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",,1,
6,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,,
7,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd",1,3,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,19,,
1,Sep 19 2022 10:29AM,246698,10,Eye Pharma Inc,1,,
2,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",10,,
3,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",3,,
4,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,19.0,NaN,NaN
1,Sep 19 2022 10:29AM,246698,10,Eye Pharma Inc,1.0,NaN,NaN
2,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",10.0,NaN,NaN
3,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",3.0,NaN,NaN
4,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,19.0,0.0,0.0
1,Sep 19 2022 10:29AM,246698,10,Eye Pharma Inc,1.0,0.0,0.0
2,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",10.0,0.0,0.0
3,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",3.0,0.0,0.0
4,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,986771,62.0,0.0,0.0
16,493370,1.0,1.0,0.0
19,246672,1.0,3.0,3.0
20,246697,19.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,986771,62.0,0.0,0.0
1,16,493370,1.0,1.0,0.0
2,19,246672,1.0,3.0,3.0
3,20,246697,19.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,62.0,0.0,0.0
1,16,1.0,1.0,0.0
2,19,1.0,3.0,3.0
3,20,19.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,62.0
1,16,Released,1.0
2,19,Released,1.0
3,20,Released,19.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Completed,0.0,0.0,3.0,0.0
Executing,0.0,1.0,3.0,0.0
Released,62.0,1.0,1.0,19.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Completed,0.0,0.0,3.0,0.0
1,Executing,0.0,1.0,3.0,0.0
2,Released,62.0,1.0,1.0,19.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Completed,0.0,0.0,3.0,0.0
1,Executing,0.0,1.0,3.0,0.0
2,Released,62.0,1.0,1.0,19.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()